In [1]:
import pandas as pd

books = pd.read_csv("./ds/cleaned_data_2[category_fixed].csv")


In [2]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "cuda")
classifier("how is this thing?")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda


[[{'label': 'surprise', 'score': 0.8574679493904114},
  {'label': 'neutral', 'score': 0.0689791664481163},
  {'label': 'anger', 'score': 0.041857894510030746},
  {'label': 'disgust', 'score': 0.016816267743706703},
  {'label': 'fear', 'score': 0.0073498310521245},
  {'label': 'sadness', 'score': 0.005906304344534874},
  {'label': 'joy', 'score': 0.0016225555445998907}]]

In [3]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548406481742859},
  {'label': 'neutral', 'score': 0.1698525846004486},
  {'label': 'sadness', 'score': 0.11640885472297668},
  {'label': 'surprise', 'score': 0.02070068195462227},
  {'label': 'disgust', 'score': 0.019100692123174667},
  {'label': 'joy', 'score': 0.01516138669103384},
  {'label': 'anger', 'score': 0.00393515033647418}]]

In [8]:
list(zip(books["description"][0].split("."), classifier(books["description"][0].split("."))))

[('A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives',
  [{'label': 'surprise', 'score': 0.7296028733253479},
   {'label': 'neutral', 'score': 0.14038555324077606},
   {'label': 'fear', 'score': 0.06816219538450241},
   {'label': 'joy', 'score': 0.047942399978637695},
   {'label': 'anger', 'score': 0.009156350046396255},
   {'label': 'disgust', 'score': 0.002628474496304989},
   {'label': 'sadness', 'score': 0.002122160978615284}]),
 (' John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers',
  [{'label': 'neutral', 'score': 0.4493706226348877},
   {'label': 'disgust', 'score': 0.2735918164253235},
   {'label': 'joy', 'score': 0.10908296704292297},
   {'label': 'sadness', 'score': 0.09362707287073135},
   {'label': 'anger', 'score': 0.04047832265496254},
   {'label': 'surprise', 'score': 0.026970142498612404},
   {'label': 'fear', 'score': 0.

In [9]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [10]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [11]:
emotion_scores

{'anger': [np.float64(0.06413362175226212),
  np.float64(0.6126182675361633),
  np.float64(0.06413362175226212),
  np.float64(0.3514835238456726),
  np.float64(0.08141248673200607),
  np.float64(0.23222509026527405),
  np.float64(0.5381847620010376),
  np.float64(0.06413362175226212),
  np.float64(0.3006700277328491),
  np.float64(0.06413362175226212)],
 'disgust': [np.float64(0.2735918164253235),
  np.float64(0.34828516840934753),
  np.float64(0.10400664806365967),
  np.float64(0.15072260797023773),
  np.float64(0.18449535965919495),
  np.float64(0.7271745800971985),
  np.float64(0.1558549851179123),
  np.float64(0.10400664806365967),
  np.float64(0.2794811725616455),
  np.float64(0.17792823910713196)],
 'fear': [np.float64(0.9281682968139648),
  np.float64(0.9425278306007385),
  np.float64(0.9723208546638489),
  np.float64(0.36070671677589417),
  np.float64(0.09504341334104538),
  np.float64(0.05136280506849289),
  np.float64(0.747427761554718),
  np.float64(0.40449613332748413),
  n

In [12]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5693/5693 [01:58<00:00, 47.93it/s]


In [13]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [14]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5688,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5689,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5690,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765,9788185300535
5691,0.064134,0.104007,0.051363,0.958549,0.915193,0.111690,0.078765,9789027712059


In [15]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [16]:
books

,Unnamed: 0,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,...,title_and_subtitle,tagged_description,simplified_category,anger,disgust,fear,joy,sadness,surprise,neutral
0,0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603
1,1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252545
2,2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765
4,4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5688,5688,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5689,5689,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5690,5690,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368110,0.078765
5691,5691,9789027712059,9027712050,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,...,The Berlin Phenomenology,9789027712059 Since the three volume edition o...,Nonfiction,0.064134,0.104007,0.051363,0.958549,0.915193,0.111690,0.078765


In [17]:
books.to_csv("./ds/cleaned_data_3[books_with_emotions].csv", index = False)